# 1.4 Sequencing Antibiotic by Shattering Them into Pieces

## Introduction to mass spectrometry
[NRPs](https://en.wikipedia.org/wiki/Nonribosomal_peptide)는 안타깝게도 ribosome을 제거해도 생성이된다. 즉, 중심이론의 예외에 해당한다. 게다가 NRPs 는 선형이 아니라 cyclic 하기 때문에 sequencing하는 것이 더 어렵다. 이를 해결하기 위해 **Mass spectrometer** 를 사용한다. Mass spectrometer는 원자의 무게를 재는 저울이다. 무게를 재는 단위로는 **Dalton** 을 사용하는데, 1 Da(Dalton) 은 양성자(proton)나 중성자(neutron) 한개의 무개를 의미한다. 예를 들면 C2H3ON은 2·12 + 3·1 + 1·16 + 1·14 = 57 Da 가 된다. 실제로 원자의 무게는 정수로 정확히 떨어지지는 않지만 단순하게 하기위해 정수만을 사용한다.

NRPs sequencing을 어렵게 하는 것은 NRPs가 linear 하지 않고 cyclic 하다는 점이다. 즉, ABC 는 CAB,BCA 와 같다. mass spectrometer는 NRPs 를 sequencing 하기 위해 이 cyclic한 peptide를 잘게 쪼갠다. 그리고, 이렇게 쪼개진 peptide의 집합을 **Experimental Spectrum** 이라고 한다.

여기서 목적은 원자들의 무게와 mass spectrometer가 쪼개놓은 조각들을 이용하여 peptide를 sequencing 하는 것이다.

## The Cyclopeptide Sequencing Problem

### 원문
For now, we will assume for simplicity that the mass spectrometer breaks the copies of a cyclic peptide at every possible two bonds, so that the resulting experimental spectrum contains the masses of all possible linear fragments of the peptide, which are called subpeptides. For example, the cyclic peptide NQEL has 12 subpeptides: N, Q, E, L, NQ, QE, EL, LN, NQE, QEL, ELN, and LNQ. We will also assume that subpeptides may occur more than once if an amino acid occurs multiple times in the peptide (e.g., ELEL also has 12 subpeptides: E, L, E, L, EL, LE, EL, LE, ELE, LEL, ELE, and LEL.

### 자의적 해석
ELEL을 잘라놓으면 E, L, E, L, EL, LE, EL, LE, ELE, LEL, ELE, LEL 와 같이 쪼개진다. 즉, cyclic 한 peptide를 1개씩 쪼개놓을 수도 있고, 2개씩 쪼개 놓을 수도 있고, 3개씩 쪼개놓을 수도 있다는 것이다.(원래의 peptide는 존재하지 않게된다.)

### Problme

**Excercise Break**: How many subpeptides does a cyclic peptide of length n have?

어떤 peptide의 길이가 주어졌을 때, 그 peptide가 쪼개질 수 있는 가지수는 몇가지인가?

In [1]:
import toolz as tz
from toolz.curried import *
from operator import *
from itertools import permutations, product, combinations_with_replacement
import math
from functools import lru_cache

In [2]:
input_data = {
    "sample": {
        "n": 31315
    },
    "test": {
        "n": int(open("data/week3/dataset_98_3.txt", "r").read().strip())
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

cur_data["n"] * (cur_data["n"] - 1)

291606852

**Theoretical Spectrum(Cyclospectrun(Peptide))** 는 Subpeptide의 무게들의 집합이다. 이 집합은 0 부터 전체 peptide 무게까지 포함하며, 작은 것부터 큰것까지 무게 순서대로 중복을 포함하는 집합이다.

**Generating Theoretical Spectrum Problem** : Generate the theoretical spectrum of cyclic peptide

**input** : an amino acid string Peptide
**output** : Cyclospectrum(peptide)

## 1.7 CS: Generating the Theoretical Spectrum of a Peptide
subpeptide의 무게는 전체 peptide의 누적 무게에서 subpeptide의 시작 amino acid 바로 이전 값까지의 누적값 을 subpeptide의 끝 amino acid 값까지의 누적값을 뺀 값과 같다.
```
LinearSpectrum(Peptide, Alphabet, AminoAcidMass)
    PrefixMass(0) ← 0
    for i ← 1 to |Peptide|
        for every symbol s in Alphabet
            if s = i-th amino acid in Peptide
                PrefixMass(i) ← PrefixMass(i − 1) + AminoAcidMass[s]
    LinearSpectrum ← a list consisting of the single integer 0
    for i ← 0 to |Peptide| − 1
        for j ← i + 1 to |Peptide|
            add PrefixMass(j) − PrefixMass(i) to LinearSpectrum
    return sorted list LinearSpectrum
```

**Code Challenge**

**Input**: An amino acid string peptide
**Output**: The linear spectrum of Peptide

In [3]:
input_data = {
    "sample": {
        "peptide": "NQEL",
    },
    "test": {
        "peptide": open("data/week3/dataset_98_4.txt").read().strip()
    }
}

cur_state = "sample"
cur_data = input_data[cur_state]

mass_table = {}
with open("data/week3/integer_mass_table.txt", "r") as f:
    for line in f:
        k, v = line.strip().split(" ")
        mass_table[k] = int(v)

def linear_spectrum(peptide, amino_acid_mass):
    mass_spectrum =[0, *accumulate(add, [amino_acid_mass[aa] for aa in peptide])]
    cyclic_mass_spectrum = mass_spectrum.copy()
    peptide_mess = last(mass_spectrum)
    for i in range(1, len(peptide)):
        for j in range(i + 1, len(peptide) + 1):
            cyclic_mass_spectrum.append(mass_spectrum[j] - mass_spectrum[i])
            if i > 0 and j < len(peptide):
                cyclic_mass_spectrum.append(peptide_mess - (mass_spectrum[j] - mass_spectrum[i]))

    return sorted(cyclic_mass_spectrum)
print(*linear_spectrum(cur_data["peptide"], mass_table))

0 113 114 128 129 227 242 242 257 355 356 370 371 484


### Stop and Think
동일한 Theoretical spectra를 가지고 있는 다른 peptides가 존재하는지 알 수 없다. 그런 peptides를 찾아 낼 수 있는가?

## 1.5 A brute force algorithm for Cyclopeptide Sequencing
$Mass(Peptide)$ 를 peptide의 전체 무게라고 했을 때, Spectrum으로 부터 $Mass(Peptide)$가 유추될 수 있으며 이를 $ParentMass(Spectrum)$라 하자. 이때, $ParentMass(Spectrum) = Max(Spectrum)$ 이다.

 Brute Force algorithm은 ParentMass(Spectrum)과 같은 무게를 가진 모든 peptide를 찾아낸 뒤에 이 peptide의 theoretical spectra가 spectrum과 일치하는지를 본다.
 ```
 BFCyclopeptideSequencing(Spectrum)
        for every peptide with Mass(Peptide) equal to ParentMass(Spectrum)
            if Spectrum = Cyclospectrum(Peptide)
                output Peptide
 ```
 
 **Counting Peptides with Given Mass Problme**: Compute the number of peptides of given mass.
 
 **input** : An integer m
 
 **output** : The number of linear peptides having integer mass m
 

In [13]:
input_data = {
    "sample": {
        "m": 1024
    },
    "test": {
        "m": open("data/week3/dataset_99_2.txt", "r").read().strip()
    }
}
cur_state = "sample"
cur_data = input_data[cur_state]


mass_table = {}
with open("data/week3/integer_mass_table.txt", "r") as f:
    for line in f:
        k, v = line.strip().split(" ")
        mass_table[int(v)] = k

def CountingMass(Mass, masslist):
    if Mass == 0: return 1, masslist
    if Mass < 57: return 0, masslist
    if Mass in masslist: return masslist[Mass], masslist
    n = 0
    for i in mass_table:
        k, masslist = CountingMass(Mass - i, masslist)
        n += k
    masslist[Mass] = n
    return n, masslist

print(CountingMass(cur_data["m"], {})[0])

$C = {\frac {y_1} {y_2}}^{\frac 1 {m_1 - m_2}}$

In [7]:
m_1, m_2 = 1024, 1225 
y_1,y_2 = 14712706211, 3740902398089

math.pow(y_1 / y_2, 1/(m_1 - m_2))



1.0279372073922501

# 1.6 A Branch-and-Bound Algorithm for Cyclopeptide Sequencing

Growing candidiate linear peptides whose theoretical spectra are consistent with the experimental spectrum. 

**STOP and Think**: 선형 peptide가 experimental spectrum에 일관된다는 것은 무슨 의미인가?

빈 문자열을 포함해, 18개 peptides(중복은 하나로)로 나열해 놓고, 이들에 amino acid를 추가해 신장을 시킨다. 단, 경우의 수가 기하급수적으로 늘어나는 것을 막기 위해 peptide 중에 experimental spectrum과 일관된 선형 peptide만 남기고 쳐낸다. 그리고 linear peptide가 Mass(spectrum)과 일치하는지를 보는 것이다.

이 방법을 **Branch and bound algorithm** 이라고 하며, 후보 peptide 수를 늘리는 것을 branching step 이라고 하고, 반대로 가망 없는 후보군을 잘라내는 것을 Bounding step 이라고 한다.

Note that sepctrum of linear peptide does not contain as many masses as the spectrum of cyclic peptide with the same amino acid seq. For instance, the theoretical spectrum of the cyclic peptide NQEL contains 14 masses (corresponding to "", N, Q, E, L, LN, NQ, QE, EL, ELN, LNQ, NQE, QEL, NQEL). However the theoretical spectrum of the linear peptide NQEL, shown below, does not contain masses correspoinding to LN, LNQ, or ELN, since these subpeptides **wrap around** the end of the linear peptide.

**cyclic** peptide NQEL는 14개의 mass를 가진다(N, Q, E, L, LN, NQ, QE, EL, ELN, LNQ, NQE, QEL, NQEL). 하지만 linear 할 경우에는 cyclic 에서 존재했던, LN, LNQ, ELN이 존재하지 않기 때문에 11개의 mass만을 가지게 된다.


**Exercise Break** : How many subpeptides does a linear peptide of given length n have?(Include the empty peptide and the entire peptide)

**input** : An integer n

**output** : The number of subpeptides of a linear peptide of length n


In [ ]:
input_data = {
    "sample": {
        "n": 4
    },
    "test": {
        "n": int(open("data/week3/dataset_100_3.txt", "r").read().strip())
    }
}

cur_state = "test"
cur_data = input_data[cur_state]

def num_subpeptide(n):
    return reduce(add, map(lambda x: n - x + 1, range(1, n+1))) + 1

print(num_subpeptide(cur_data["n"]))